In [231]:
import pandas as pd
import numpy as np
from collections import Counter

In [232]:
data = pd.read_csv("diabetes.csv")
data.head()
data.columns[1]
data.info()
data1= data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [233]:
y = data.Outcome
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

Begin the tree with the root node with all the data set<br>
Find the best attribute in the dataset <br>
Divide the Sinto subset that contains possible values <br>
Create the decision tree node which contains possible values <br>
Recursively make new decision tree using the subset of new node in step 3<br>

In [234]:
x=data.drop(['Outcome'], axis=1)
x

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [235]:

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [236]:
x_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
663,9,145,80,46,130,37.9,0.637,40
712,10,129,62,36,0,41.2,0.441,38
161,7,102,74,40,105,37.2,0.204,45
509,8,120,78,0,0,25.0,0.409,64
305,2,120,76,37,105,39.7,0.215,29
...,...,...,...,...,...,...,...,...
645,2,157,74,35,440,39.4,0.134,30
715,7,187,50,33,392,33.9,0.826,34
72,13,126,90,0,0,43.4,0.583,42
235,4,171,72,0,0,43.6,0.479,26


In [237]:
class Node:
    def __init__(self, feature=None, threshold=None, children = None, gain = None,*,value= None) -> None:
        self.feature = feature
        self.threshold = threshold#?
        self.children = children
        self.value=value
        self.gain = gain

    def is_leaf_node(self):
        return self.value is not None


In [238]:

class DecisionTree:
    def __init__(self, min_samples_split=0, max_depth=70, childSize=5, n_features=None):
        self.minSample = min_samples_split
        self.maxDepth = max_depth
        self.maxFeature = n_features
        self.node = None
        self.childSize = childSize

    def fit(self, X, y):
        self.maxFeature = X.shape[1] if not self.maxFeature else min(
            X.shape[1], self.maxFeature)
        self.node = self.DTree(X, y)

    def DTree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth >= self.maxDepth or n_labels == 1 or n_samples < self.minSample):
            leaf_value = self.mostLable(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.maxFeature, replace=False)

        best_feature, best_thresh, best_gain = self.findingFeature(
            X, y, feat_idxs)

        child_idxs = self.split(X[:, best_feature], best_thresh)
        child_idxs = np.array(child_idxs)
        children = []
        for i in range(len(child_idxs)):
            # print(f'child:{child_idxs}')
            if y[child_idxs[i]].size == 0:
                print("child_idx is empty")
            children.append(self.DTree(
                X[child_idxs[i], :], y[child_idxs[i]], depth+1))
        node = Node(best_feature, best_thresh, children, best_gain)

        return node

# land deliyd
    def findingFeature(self, X, y, feat_idxs):
        # best_gain = -1
        np.random.seed()
        # print(f'best split y size {y.size}')
        split_idx = None
        split_threshold = []
        best_gains = -1
        all_gain = []
        for feat_idx in feat_idxs:
            threshold = []
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            temp = np.unique(thresholds)
            temp.sort()
            length = temp.size
            everyPart = length / self.childSize
            for i in range(self.childSize - 1):
                num = int((i+1) * everyPart)
                if temp[num] not in threshold:
                    threshold.append(temp[num])
            gain = self.ig(y, X_column, threshold)
            all_gain.append((gain, feat_idx))
            if gain > best_gains:
                best_gains = gain
                split_idx = feat_idx
                split_threshold = threshold
        print(all_gain)
        return split_idx, split_threshold, best_gains


# split_idx -> best feature


    def ig(self, y, X_column, threshold):
        parent_entropy = self.entropy(y)
        children = self.split(X_column, threshold)
        # chil = 0
        # for i in children:
        #     chil += len(i)
        for i in children:
            if len(i) == 0:
                return 0
        n = len(y)
        n_l = []
        e_l = []
        for i in children:
            n_l.append(len(i))

        for i in children:
            e_l.append(self.entropy(y[i]))

        child_entropy = 0
        for i in range(len(n_l)):
            child_entropy += (n_l[i]/n) * e_l[i]
        # calculate the IG
        information_gain = parent_entropy - child_entropy
        # if information_gain < 0:
        print(f'-------------------entropy')
        print(f'len{X_column.size}')
        print(
            f'information gain-> parent: {parent_entropy} child: {child_entropy}')
        print(f'parentlen: {len(y)} child')
        print(f'e_l -> {e_l}')
        print(f'n_l -> {n_l}')
        print(f'-------------------end')

        return information_gain

    def split(self, X_column, split_thresh):
        #left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        # indx of which argument is smaller than split_thresh and flatten give you one list
        #right_idxs = np.argwhere(X_column > split_thresh).flatten()
        answer = []
        # temp = X_column
        split_thresh.sort()
        # if len(np.argwhere(X_column > split_thresh[0]).flatten()) == 0:
        if np.any(np.argwhere(X_column < split_thresh[0])):
            answer.append(np.argwhere(X_column < split_thresh[0]).flatten())
        count = 1
        while count < len(split_thresh):
            # print(f'-------------------------------------{np.argwhere(X_column >= split_thresh).flatten()}')
            temp = np.argwhere(X_column >= split_thresh[count - 1]).flatten()
            temp2 = np.argwhere(X_column < split_thresh[count]).flatten()
            val = [val for val in temp if val in temp2]
            if val != []:
                # print(val)
                answer.append(np.array(val))

            count += 1

        if np.any(np.argwhere(X_column >= split_thresh[len(split_thresh) - 1])):
            answer.append(np.argwhere(
                X_column >= split_thresh[len(split_thresh) - 1]).flatten())
        return answer

    def entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])

    def mostLable(self, y):
        counter = Counter(y)
        # print(f'most common{y}')
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self.findingAnswer(x, self.node) for x in X])

    def findingAnswer(self, x, node):
        if node.is_leaf_node():
            return node.value
        for thr, i in zip(node.threshold, range(len(node.children))):
            if x[node.feature] <= thr:
                # TODO SORT TRESHOLDS
                return self.findingAnswer(x, node.children[i])
        return self.findingAnswer(x, node.children[len(node.children) - 1])


In [239]:
x = x.to_numpy()
y = y.to_numpy()

In [240]:
x_train , x_test, y_train, y_test = tarin = train_test_split(
    x, y , test_size = 0.2, random_state = 1234
)
# print(type(X_train))

In [241]:
clf = DecisionTree()
clf.fit(x_train, y_train)
prediction = clf.predict(x_test)

-------------------entropy
len614
information gain-> parent: 0.6455099710688985 child: 0.6252416464798006
parentlen: 614 child
e_l -> [0.6594687210865972, 0.4916841165135498, 0.6482791673644765, 0.6910836447921442, 0.6869615765973234]
n_l -> [62, 129, 296, 109, 18]
-------------------end
-------------------entropy
len614
information gain-> parent: 0.6455099710688985 child: 0.5897455605672173
parentlen: 614 child
e_l -> [0.516998183548264, 0.6907114543575098, 0.6809991432654108, 0.6829081047004717, 0.48691270946460224]
n_l -> [325, 129, 90, 49, 21]
-------------------end
-------------------entropy
len614
information gain-> parent: 0.6455099710688985 child: 0.637979818882073
parentlen: 614 child
e_l -> [0.6313727800636019, 0.5728921332162746, 0.6778978109318121, 0.6767267229246678, 0.6581099875431143]
n_l -> [230, 127, 155, 83, 19]
-------------------end
-------------------entropy
len614
information gain-> parent: 0.6455099710688985 child: 0.6199445593000178
parentlen: 614 child
e_l -> [

/tmp/ipykernel_20699/3054250141.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  child_idxs = np.array(child_idxs)


-------------------entropy
len4
information gain-> parent: 0.6931471805599453 child: 0.0
parentlen: 4 child
e_l -> [-0.0, -0.0, -0.0, -0.0]
n_l -> [1, 1, 1, 1]
-------------------end
-------------------entropy
len4
information gain-> parent: 0.6931471805599453 child: 0.0
parentlen: 4 child
e_l -> [-0.0, -0.0, -0.0, -0.0]
n_l -> [1, 1, 1, 1]
-------------------end
-------------------entropy
len4
information gain-> parent: 0.6931471805599453 child: 0.0
parentlen: 4 child
e_l -> [-0.0, -0.0, -0.0]
n_l -> [1, 1, 1]
-------------------end
-------------------entropy
len4
information gain-> parent: 0.6931471805599453 child: 0.0
parentlen: 4 child
e_l -> [-0.0, -0.0, -0.0, -0.0]
n_l -> [1, 1, 1, 1]
-------------------end
-------------------entropy
len4
information gain-> parent: 0.6931471805599453 child: 0.0
parentlen: 4 child
e_l -> [-0.0, -0.0, -0.0]
n_l -> [1, 1, 1]
-------------------end
[(0.34657359027997264, 4), (0.6931471805599453, 3), (0.6931471805599453, 1), (0.6931471805599453, 6), (

In [242]:
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred)/len(y_test)

In [243]:
acc = accuracy(y_test, prediction)
print(acc)

0.6753246753246753


In [244]:
# dataArr = np.array(data)
def dfs(tree:Node, depth = 0):
    if tree != None:
        if tree.children != None:
            print(depth * '\t' + f'threshold:{tree.threshold} and feature:{data1.columns[tree.feature]}')
            for child in tree.children:
                # print(depth * '\t' + f'value: {child.value} {child.threshold} and {child.feature}')
                dfs(child, depth+1)
        else:
            print(depth * '\t' + f'value: {tree.value}')

        
            
    # print(tree.root)
dfs(clf.node)

threshold:[89.0, 116.0, 142.0, 169.0] and feature:Glucose
	threshold:[16.0, 23.0, 29.0, 37.0] and feature:SkinThickness
		value: 0
		threshold:[22.0, 25.0, 28.0, 32.0] and feature:Age
			value: 0
			value: 0
			value: 0
			value: 0
			threshold:[24.4, 29.0] and feature:BMI
				value: 0
				value: 1
		value: 0
		threshold:[24.0, 26.0, 28.0, 37.0] and feature:Age
			value: 0
			value: 0
			value: 1
			value: 0
			threshold:[5.0, 6.0, 12.0] and feature:Pregnancies
				value: 1
				value: 0
				value: 1
		value: 0
	threshold:[29.0, 37.0, 45.0, 56.0] and feature:Age
		threshold:[23.6, 27.1, 31.2, 37.4] and feature:BMI
			threshold:[41.0, 46.0, 51.0, 82.0] and feature:Insulin
				value: 0
				value: 0
				threshold:[96.0, 100.0, 107.0] and feature:Glucose
					value: 0
					value: 0
					value: 1
				value: 0
				value: 0
			value: 0
			value: 0
			threshold:[62.0, 68.0, 74.0, 78.0] and feature:BloodPressure
				value: 0
				threshold:[64.0, 74.0, 87.0, 99.0] and feature:Insulin
					value: 